# Indeed Scraper

Scrape Indeed for job postings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from plotnine import *

## Version 1

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# wd.get("https://www.webite-url.com")

In [ ]:
driver.get('https://indeed.com')

In [ ]:
# enter search details here

# search grocery clerk
search_job = driver.find_element_by_xpath("//input[@id='text-input-what']") 
search_job.send_keys(['grocery clerk'])

search_loc = driver.find_element_by_xpath("//input[@id='text-input-where']")
search_loc.send_keys(['Los Angeles, CA'])

# find and click "Find Jobs" button
initial_search_button = driver.find_element_by_xpath("//form[@id='whatWhereFormId']/div[3]")
initial_search_button.click()

In [ ]:
#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 

In [ ]:
titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]
dateposted=[]

This is the part to fix !! 

In [ ]:

for i in range(0,2): #then 20
    
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"mosaic-provider-jobcards")]')
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
    
    #try:
    #    next_page = driver.find_element_by_xpath("//span[@class='pn']")
        #driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
    #    next_page.click()

    #except:
    #    next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="pn"]')
    #    next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        
    
    print("Page: {}".format(str(i+2)))

Page: 2
Page: 3


In [ ]:
# get job descriptions
descriptions=[]
for link in links:
    
    driver.get(link)
    jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)

In [ ]:
# put in a dataframe
df_da=pd.DataFrame()
df_da['Title']=titles
df_da['Company']=companies
df_da['Location']="Los Angeles, CA"
df_da['Link']=links
df_da['Review']=reviews
df_da['Salary']=salaries
df_da['Description']=descriptions

In [ ]:
titles

[]

## Version 2

In [ ]:
import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

In [ ]:
def find_jobs_from(website, job_title, location, desired_characs, filename="results.csv"):    
    """
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'Indeed' or 'CWjobs')
        - Job_title
        - Location
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: to specify the filename and format of the output.
            Default is .xls file called 'results.xls'
    """
    
    if website == 'Indeed':
        job_soup = load_indeed_jobs_div(job_title, location)
        jobs_list, num_listings = extract_job_information_indeed(job_soup, desired_characs)
    
    save_jobs_to_csv(jobs_list, filename)
 
    print('{} new job postings retrieved from {}. Stored in {}.'.format(num_listings, 
                                                                          website, filename))

In [ ]:
def save_jobs_to_excel(jobs_list, filename):
    jobs = pd.DataFrame(jobs_list)
    jobs.to_excel(filename)

def save_jobs_to_csv(jobs_list, filename):
    jobs = pd.DataFrame(jobs_list)
    jobs.to_csv(filename)

In [ ]:
def load_indeed_jobs_div(job_title, location):
    getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date'}
    url = ('https://www.indeed.com/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

def extract_job_information_indeed(job_soup, desired_characs):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    cols = []
    extracted_info = []
    
    
    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)                    
    
    if 'companies' in desired_characs:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeed(job_elem))
        extracted_info.append(links)
    
    if 'date_listed' in desired_characs:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return company

def extract_link_indeed(job_elem):
    link = job_elem.find('a')['href']
    link = 'www.Indeed.com/' + link
    return link

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date

In [ ]:
find_jobs_from('Indeed', 'Grocery Clerk', 'Los Angeles, CA', ['titles'], filename="results.csv")

15 new job postings retrieved from Indeed. Stored in results.csv.


KeyboardInterrupt: ignored

## Version 3

In [ ]:
pip install jobs_scraper

  Created wheel for jobs-scraper: filename=jobs_scraper-0.0.10-cp36-none-any.whl size=4848 sha256=9130c5f8696d05f5f62af55ac9f091c26def1f99ad5f31ea116e5bed04d185e5
  Stored in directory: /root/.cache/pip/wheels/86/dd/7f/7afacdf410d538af509942447b53d3b74b390da2793038d745
Successfully built jobs-scraper


In [ ]:
# https://pypi.org/project/jobs-scraper/
from jobs_scraper import JobsScraper

In [ ]:
scraper = JobsScraper(country="US", position="Grocery Clerk", 
                      location="Los Angeles, CA", pages=3,
                      max_delay = 5)

df = scraper.scrape()

Scraping Grocery - Clerk...
Scraping Ralphs Retail Clerk (9th & Flower)...
Scraping Grocery Clerk...
Scraping Food 4 Less Retail Clerk (Olympic & Hoover)...
Scraping Grocery Clerk...
Scraping Grocery Clerk...
Scraping Ralphs Retail Clerk (Wiltern)...
Scraping Grocery Clerk...
Scraping Night Crew Clerk (overnight shift)...
Scraping Food 4 Less Retail Clerk (Olympic & Colada)...
Scraping Food 4 Less Retail Clerk (Athens)...
Scraping Courtesy Clerk...
Scraping Night Crew Clerk (graveyard shift) SK# 6 - Van Nuys, CA...
Scraping COMMISSARY SUPPORT CLERK...
Scraping Ralphs Retail Clerk (Colorado)...
Scraping Grocery General Merchandise Clerk...
Scraping Ralphs Retail Clerk (Wilshre/Hauser)...
Scraping Ralphs Retail Clerk (Coldwater)...
Scraping Ralphs Retail Clerk (Slauson & Crenshaw)...
Scraping Food 4 Less Retail Clerk (Watts)...
Scraping International Grocery Clerk...
Scraping Ralphs Retail Clerk (Olympic & Cloverfield)...
Scraping Ralphs Retail Clerk (San Fernando & Alameda)...
Scraping 

In [ ]:
df.head()

,title,location,company,summary,salary,url
0,Grocery - Clerk,"Los Angeles, CA",Bristol Farms and Lazy Acres Natural Market,"Stocks shelves, dairy case and other areas of ...",None,https://indeed.com/rc/clk?jk=c8545fc5aae35dad&...
1,Ralphs Retail Clerk (9th & Flower),"Los Angeles, CA 90015 (Downtown area)",Ralphs,Assist customers at checkout and bag groceries...,None,https://indeed.com/rc/clk?jk=0f972951eff3bae3&...
2,Grocery Clerk,"Glendale, CA 91201",Super King Market,This role will be responsible for unloading an...,$14 - $16 an hour,https://indeed.com/rc/clk?jk=322949d6c1c2d0c8&...
3,Food 4 Less Retail Clerk (Olympic & Hoover),"Los Angeles, CA 90006 (Harvard Heights area)",Food 4 Less,Assist customers at checkout and bag groceries...,None,https://indeed.com/rc/clk?jk=8800dac325a156d5&...
4,Grocery Clerk,"Los Angeles, CA 90016 (West Adams area)",SK MARKET INC,Open cartons and price items accurately; keep ...,None,https://indeed.com/rc/clk?jk=b300e8f9df8bb053&...


## Version 4

In [ ]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

The Indeed URL has some structure to it that we can exploit to make the scraping easier.

In [ ]:
# define URL template
url_template = "https://www.indeed.com/jobs?q={}&l={}&start={}"

# page 1 https://www.indeed.com/jobs?q=grocery+clerk&l=Los+Angeles%2C+CA&start=0
# page 2 https://www.indeed.com/jobs?q=grocery+clerk&l=Los+Angeles%2C+CA&start=10

In [ ]:
# Input
city_name = 'los+angeles'
state = 'CA'

# loc to be searched
location = city_name + '%2C+' + state 

print(location)

los+angeles%2C+CA


In [ ]:
max_results = 100

i = 0
results = [] 
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])


# Version 5

In [72]:
import indeed_scraper

In [ ]:
scraper = indeed_scraper.JobsScraper(country="US", position="science", 
                      location="Los Angeles, CA", pages=1,
                      max_delay = 3)

df = scraper.scrape()

df.head()

In [ ]:
df.tail()

,title,location,company,summary,salary,date_posted,date_scraped,url
10,Basic Science Research Technician,"Los Angeles, CA 90059",Charles R. Drew University,Familiar with molecular biology techniques/sci...,None,8 days ago,2021-01-29,https://indeed.com/rc/clk?jk=10ddf1f8b8686c85&...
11,Laboratory Scientist II,"Chino, CA 91708",Inland Empire Utilities Agency,Graduation from an accredited four-year colleg...,$40.73 - $49.62 an hour,17 days ago,2021-01-29,https://indeed.com/rc/clk?jk=d997a22095af700e&...
12,Microbiology Laboratory Technician 1,"Torrance, CA 90503",ALS Environmental,College courses in micro or life sciences.The ...,None,Today,2021-01-29,https://indeed.com/rc/clk?jk=d29885943b97f632&...
13,Research Engineering Assistant,"Pasadena, CA 91125 (South area)",California Institute of Technology,"Reporting to Professor Richard Flagan, the Res...",None,Just posted,2021-01-29,https://indeed.com/rc/clk?jk=8cab79ac68cca1af&...
14,"Laboratory Associate I,II,III","Irvine, CA 92618 (East Industrial Complex area)",ResearchDx,Ability to lift 15-25 pounds and stand for pro...,None,Today,2021-01-29,https://indeed.com/rc/clk?jk=5e50dee90e1c413d&...
